In [1]:
import joblib
import sys

splits = joblib.load('/workspace/Code/tutorials/pathology/hovernet/splits.dat')
def prepare_data(data_dir, fold, splits):
    images = np.load(os.path.join(data_dir, "images.npy"))
    labels = np.load(os.path.join(data_dir, "labels.npy"))

    data = [
        {
            "image": image,
            "image_meta_dict": {"original_channel_dim": -1},
            "label": label,
            "label_meta_dict": {"original_channel_dim": -1},
        }
        for image, label in zip(images, labels)
    ]
    train_data = [data[i] for i in splits[fold]['train']]
    valid_data = [data[i] for i in splits[fold]['valid']]
    
    return train_data, valid_data

In [11]:
import cv2
import numpy as np

from scipy.ndimage import filters, measurements
from scipy.ndimage.morphology import (
    binary_dilation,
    binary_fill_holes,
    distance_transform_cdt,
    distance_transform_edt,
)

from skimage.segmentation import watershed

import warnings

def get_bounding_box(img):
    """Get bounding box coordinate information."""
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    # due to python indexing, need to add 1 to max
    # else accessing will be 1px in the box, not out
    rmax += 1
    cmax += 1
    return [rmin, rmax, cmin, cmax]

def remove_small_objects(pred, min_size=64, connectivity=1):
    """Remove connected components smaller than the specified size.
    This function is taken from skimage.morphology.remove_small_objects, but the warning
    is removed when a single label is provided. 
    Args:
        pred: input labelled array
        min_size: minimum size of instance in output array
        connectivity: The connectivity defining the neighborhood of a pixel. 
    
    Returns:
        out: output array with instances removed under min_size
    """
    out = pred

    if min_size == 0:  # shortcut for efficiency
        return out

    if out.dtype == bool:
        selem = ndimage.generate_binary_structure(pred.ndim, connectivity)
        ccs = np.zeros_like(pred, dtype=np.int32)
        ndimage.label(pred, selem, output=ccs)
    else:
        ccs = out

    try:
        component_sizes = np.bincount(ccs.ravel())
    except ValueError:
        raise ValueError(
            "Negative value labels are not supported. Try "
            "relabeling the input with `scipy.ndimage.label` or "
            "`skimage.morphology.label`."
        )

    too_small = component_sizes < min_size
    too_small_mask = too_small[ccs]
    out[too_small_mask] = 0

    return out

def noop(*args, **kargs):
    pass


warnings.warn = noop


####
def __proc_np_hv(pred):
    """Process Nuclei Prediction with XY Coordinate Map.
    Args:
        pred: prediction output, assuming 
              channel 0 contain probability map of nuclei
              channel 1 containing the regressed X-map
              channel 2 containing the regressed Y-map
    """
#     pred = np.array(pred, dtype=np.float32)

#     blb_raw = pred[..., 0]
#     h_dir_raw = pred[..., 1]
#     v_dir_raw = pred[..., 2]

#     # processing
#     blb = np.array(blb_raw >= 0.5, dtype=np.int32)

    blb_raw = pred[HoVerNetBranch.NP.value]
    h_dir_raw = pred[HoVerNetBranch.HV.value][0,...]
    v_dir_raw = pred[HoVerNetBranch.HV.value][1,...]
    blb = Activations(sigmoid=False, softmax=True)(blb_raw)
    blb = AsDiscrete(argmax=True)(blb)

    blb = blb.detach().cpu()
    h_dir_raw = h_dir_raw.detach().cpu().numpy()
    v_dir_raw = v_dir_raw.detach().cpu().numpy()
    blb = measurements.label(blb)[0]
    blb = remove_small_objects(blb, min_size=10)
    blb[blb > 0] = 1  # background is 0 already

    h_dir = cv2.normalize(
        h_dir_raw, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F
    )
    v_dir = cv2.normalize(
        v_dir_raw, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F
    )

    sobelh = cv2.Sobel(h_dir, cv2.CV_64F, 1, 0, ksize=21)
    sobelv = cv2.Sobel(v_dir, cv2.CV_64F, 0, 1, ksize=21)

    sobelh = 1 - (
        cv2.normalize(
            sobelh, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F
        )
    )
    sobelv = 1 - (
        cv2.normalize(
            sobelv, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F
        )
    )

    overall = np.maximum(sobelh, sobelv)
    overall = overall - (1 - blb)
    overall[overall < 0] = 0

    dist = (1.0 - overall) * blb
    ## nuclei values form mountains so inverse to get basins
    dist = -cv2.GaussianBlur(dist, (3, 3), 0)

    overall = np.array(overall >= 0.8, dtype=np.int32)

    marker = blb - overall
    marker[marker < 0] = 0
    marker = binary_fill_holes(marker).astype("uint8")
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    marker = cv2.morphologyEx(marker, cv2.MORPH_OPEN, kernel)
    marker = measurements.label(marker)[0]
    marker = remove_small_objects(marker, min_size=10)

    proced_pred = watershed(dist, markers=marker, mask=blb)
    proced_pred[proced_pred>0] = 1
    return proced_pred



In [37]:
model0p = "/workspace/Code/tutorials/pathology/hovernet/runs/ngc-exp/Oct26_15-14-17_3577198_0/best_metric_model0.pth"
model1p = "/workspace/Code/tutorials/pathology/hovernet/runs/ngc-exp/Oct26_15-15-55_3577198_1/best_metric_model1.pth"
model2p = "/workspace/Code/tutorials/pathology/hovernet/runs/ngc-exp/Oct26_15-17-07_3577198_2/best_metric_model2.pth"

model_list = [model0p, model1p, model2p]
import sys
sys.path.append('/workspace/Code/tutorials/pathology/hovernet/transforms')
sys.path.append('/workspace/Code/tutorials/pathology/hovernet/loss')
from loss import HoVerNetLoss
from transforms import (
    GenerateWatershedMaskd,
    GenerateInstanceBorderd,
    GenerateDistanceMapd,
    GenerateWatershedMarkersd,
    Watershedd,
    GenerateInstanceContour,
    GenerateInstanceCentroid,
    GenerateInstanceType,
    GenerateInstanceCentroid, 
    GenerateInstanceContour, 
    GenerateInstanceType,
)

import os
import time
import torch
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import pandas as pd
from monai.data import DataLoader, decollate_batch, Dataset, CacheDataset
from monai.metrics import DiceMetric
from monai.networks.nets import HoVerNet
from monai.transforms import (
    Activations,
    AsDiscrete,
    AsDiscreted,
    Compose,
    ScaleIntensityRanged,
    CastToTyped,
    Lambdad,
    SplitDimd,
    EnsureChannelFirstd,
    ComputeHoVerMapsd,
    RandFlipd,
    RandRotate90d,
    RandGaussianSmoothd,
    FillHoles,
    BoundingRect,
    CenterSpatialCropd,
    SaveImage,
    GaussianSmooth,
)

from monai.utils import set_determinism, convert_to_tensor
from monai.utils.enums import HoVerNetBranch
from monai.visualize import plot_2d_or_3d_image
from sklearn.model_selection import StratifiedShuffleSplit
            

def post_process(output, device, return_binary=True, return_centroids=False, output_classes=None):
    post_trans_seg = Compose([
        GenerateWatershedMaskd(keys=HoVerNetBranch.NP.value, softmax=True),
        GenerateInstanceBorderd(keys='mask', hover_map_key=HoVerNetBranch.HV, kernel_size=3),
        GenerateDistanceMapd(keys='mask', border_key='border', smooth_fn=GaussianSmooth()),
        GenerateWatershedMarkersd(keys='mask', border_key='border', threshold=0.99, radius=2, postprocess_fn=FillHoles()),
        Watershedd(keys='dist', mask_key='mask', markers_key='markers')
    ])
    if HoVerNetBranch.NC.value in output.keys():
        type_pred = Activations(softmax=True)(output[HoVerNetBranch.NC.value])
        type_pred = AsDiscrete(argmax=True)(type_pred)

    pred_inst_dict = post_trans_seg(output)
    pred_inst = pred_inst_dict['dist']

    inst_id_list = np.unique(pred_inst)[1:]  # exclude background
inst_info
    inst_info_dict = None
    if return_centroids:
        inst_id_list = np.unique(pred_inst)[1:]  # exclude background
        inst_info_dict = {}
        for inst_id in inst_id_list:
            inst_map = pred_inst == inst_id
            inst_bbox = BoundingRect()(inst_map)
            inst_map = inst_map[:, inst_bbox[0][0]: inst_bbox[0][1], inst_bbox[0][2]: inst_bbox[0][3]]
            offset = [inst_bbox[0][2], inst_bbox[0][0]]
            inst_contour = GenerateInstanceContour()(inst_map.squeeze(), offset)
            inst_centroid = GenerateInstanceCentroid()(inst_map, offset)
            if inst_contour is not None:
                inst_info_dict[inst_id] = {  # inst_id should start at 1
                    "bounding_box": inst_bbox,
                    "centroid": inst_centroid,
                    "contour": inst_contour,
                    "type_probability": None,
                    "type": None,
                }

    if output_classes is not None:
        for inst_id in list(inst_info_dict.keys()):
            inst_type, type_prob = GenerateInstanceType()(
                bbox=inst_info_dict[inst_id]["bounding_box"], type_pred=type_pred, seg_pred=pred_inst, instance_id=inst_id)
            inst_info_dict[inst_id]["type"] = inst_type
            inst_info_dict[inst_id]["type_probability"] = type_prob

    pred_inst = convert_to_tensor(pred_inst, device=device)
    if return_binary:
        pred_inst[pred_inst > 0] = 1
    return (pred_inst, inst_info_dict, pred_inst_dict)

def main(data_dir, fold):

    test_transforms = Compose(
        [
            EnsureChannelFirstd(keys=("image", "label"), channel_dim=-1),
            SplitDimd(keys="label", output_postfixes=["inst", "type"]),
            ComputeHoVerMapsd(keys="label_inst"),
            CastToTyped(keys=["image", "label_inst", "label_type", "hover_label_inst"], dtype=torch.float32),
            Lambdad(keys="label", func=lambda x: x[1: 2, ...] > 0),
            AsDiscreted(keys=["label", "label_type"], to_onehot=[2, 7]),
            CenterSpatialCropd(keys=["label", "label_inst", "label_type", "hover_label_inst"], roi_size=(164,164)),
            ScaleIntensityRanged(keys=["image"], a_min=0.0, a_max=255.0, b_min=0.0, b_max=1.0, clip=True),
        ]
    )

        
    post_process_0 = Compose(
    [
        Activations(sigmoid=False, softmax=True),
        
    ])
    
    train_data, valid_data = prepare_data(data_dir, fold, splits)
    test_ds = Dataset(data=valid_data, transform=test_transforms)
    test_loader = DataLoader(test_ds, batch_size=8, num_workers=4, pin_memory=True)

    dice_metric = DiceMetric(include_background=False, reduction="mean")
    device = torch.device("cuda:1")
    model = HoVerNet(
        mode="fast",
        in_channels=3,
        out_classes=7,
        act=("relu", {"inplace": True}),
        norm="batch",
        dropout_prob=0.2,
    ).to(device)

#     model.load_state_dict(torch.load(os.path.join(data_dir, "best_metric_model.pth")))
    model.load_state_dict(torch.load(model_list[fold]))

    model.eval()
    with torch.no_grad():
        for test_data in test_loader:
            test_inputs, test_label, test_label_type, test_hover_map = (
                    test_data["image"].to(device),
                    test_data["label"].to(device),
                    test_data["label_type"].to(device),
                    test_data["hover_label_inst"].to(device),
                )

            test_outputs = model(test_inputs)
#             test_outputs = [torch.tensor(__proc_np_hv(i)).to(device) for i in decollate_batch(test_outputs)]
            test_outputs = [AsDiscrete(threshold=0.5)(post_process_0(i[HoVerNetBranch.NP.value])[1:2,...]) for i in decollate_batch(test_outputs)]
            test_label = [i for i in decollate_batch(test_label)]
    
            # compute metric for current iteration
            dice_metric(y_pred=test_outputs, y=test_label)

        metric = dice_metric.aggregate().item()
        # aggregate the final mean dice result
        print("evaluation metric:", metric)
        # reset the status
        dice_metric.reset()



In [41]:
data_dir = "/workspace/Data/Lizard/Prepared"
main(data_dir, 0)

evaluation metric: 0.8250155448913574


In [ ]:
fold0: 
    evaluation metric: 0.7615175843238831  # 0.6
    evaluation metric: 0.8022753596305847  # 0.65
    evaluation metric: 0.8130292296409607  # 0.7
    evaluation metric: 0.8172582387924194  # 0.75
    evaluation metric: 0.8192526698112488  # 0.8
    evaluation metric: 0.8202919960021973  # 0.85
    evaluation metric: 0.8210670351982117  # 0.9
    evaluation metric: 0.8215556144714355  # 0.95
    evaluation metric: 0.8216920495033264  # 0.99

fold1: 
    evaluation metric: 0.8130324482917786
    
fold2:
    evaluation metric: 0.8064720630645752

In [ ]:
0.8250155448913574
0.8159043788909912
0.8085355758666992

0.8163